<a href="https://colab.research.google.com/github/StephenSpicer/Unit_02_Clone/blob/master/Stephen_Lupsha_LS_DS_212_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [ ]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv',
                  parse_dates = ['created'],
                  index_col='created')

assert df.shape == (49352, 33)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [ ]:
df.head(10)

,bathrooms,bedrooms,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,amenities_score
created,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-06-24 07:54:24,1.5,3,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-06-12 12:19:27,1.0,2,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5
2016-04-17 03:26:41,1.0,1,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
2016-04-18 02:22:02,1.0,1,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
2016-04-28 01:32:41,1.0,4,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2016-04-19 04:24:47,2.0,4,,West 18th Street,40.7429,-74.0028,7995,350 West 18th Street,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-04-27 03:19:56,1.0,2,Stunning unit with a great location and lots o...,West 107th Street,40.8012,-73.9660,3600,210 West 107th Street,low,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
2016-04-13 06:01:42,2.0,1,"This huge sunny ,plenty of lights 1 bed/2 bath...",West 21st Street,40.7427,-73.9957,5645,155 West 21st Street,low,1,0,1,0,1,1,0,0,0,1,1,0,1,0,0,0,0,0,1,0,0,0,0,0,8
2016-04-20 02:36:35,1.0,1,<p><a website_redacted,Hamilton Terrace,40.8234,-73.9457,1725,63 Hamilton Terrace,medium,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4


In [ ]:
# shamelessly pasting my own code from homework. 
df_amenities = df.copy()
#copy
df_amenities.drop(['description','bathrooms', 'bedrooms', 'display_address', 'latitude', 'longitude', 'price', 'street_address', 'interest_level'], axis=1, inplace=True)
#drop the non-boolean variables. 

df_amenities["amenities_score"] = df_amenities.sum(axis=1)
# sum it up into a new column. 
df["amenities_score"] = df_amenities['amenities_score']
df.head()

,bathrooms,bedrooms,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,amenities_score
created,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-06-24 07:54:24,1.5,3,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-06-12 12:19:27,1.0,2,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5
2016-04-17 03:26:41,1.0,1,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
2016-04-18 02:22:02,1.0,1,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
2016-04-28 01:32:41,1.0,4,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


Ok, we have some basic EDA and we have an amenities score. Now we can do the work of splitting / training / testing that along something else like bedrooms or bathrooms. nbd. lets go. I have a feeling I'm gonna want to clean up the Datetime Index there but I'll get some typing done below first. 

In [ ]:
# Target Vector and Feature matrix. 

X = df[['amenities_score']]
y = df['price']


In [ ]:
# gonna save this for later 
# X = df[['amenities_score', 'bedrooms']]
# y = df['price']


Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression

In [ ]:
cutoff = df.index.month == 6

mask = X.index.month != 6

X_train, y_train = X.loc[mask], y.loc[mask]
X_test, y_test = X.loc[~mask], y.loc[~mask]

In [ ]:
print(X.shape)
print(X_train.shape)
print(X_test.shape)

(48817, 1)
(31844, 1)
(16973, 1)


In [ ]:
print(y.shape)
print(y_train.shape)
print(y_test.shape)

(48817,)
(31844,)
(16973,)


In [ ]:


y_pred = [y_train.mean()] * len(y_train) #pretty sure this is my training data
print('Mean Price for an NYC rental from this dataset:', y_train.mean())
print('Baseline Mean Absolute Error : ', mean_absolute_error(y_train,  y_pred))

Mean Price for an NYC rental from this dataset: 3575.604007034292
Baseline Mean Absolute Error :  1201.8811133682555


Alright, awesome, we have those two now. We have "Established a baseline" as they say. 

In [ ]:
# Now we build the model. 

# predictor class, I think we already had this above but what the heck. 
from sklearn.linear_model import LinearRegression

# Instantiate your model (class --> object) - is instantiation a word?
model_1 = LinearRegression()

# Step 3: Fit your TRAINING data to the model
model_1.fit(X_train, y_train);

In [ ]:
# Now (unless I am very much mistaken) we want to find the MAE for our MODEL as opposed to our BASELINE which we already did. 

print('Training MAE:', mean_absolute_error(y_train, model_1.predict(X_train)))
print('Test MAE:', mean_absolute_error(y_test, model_1.predict(X_test)))

Training MAE: 1139.7197144621534
Test MAE: 1127.2521918670423


DANG. that is barely an improvement. moving on for now, let's see if the RMSE is any better... fingers crossed over here. 

In [ ]:
from sklearn.metrics import mean_squared_error
# remember to set squared to false!!!
print('Training RMSE:', mean_squared_error(y_train, model_1.predict(X_train), squared=False))
print('Test RMSE:', mean_squared_error(y_test, model_1.predict(X_test), squared=False))

Training RMSE: 1679.6910460472839
Test RMSE: 1675.8140200529506


In [ ]:
from sklearn.metrics import r2_score

print('Training R^2:', r2_score(y_train, model_1.predict(X_train)))
print('Test R^2:', r2_score(y_test, model_1.predict(X_test)))

Training R^2: 0.09135704663981226
Test R^2: 0.0964178209287424


In [ ]:
print('Baseline Mean Absolute Error : ', mean_absolute_error(y_train,  y_pred))
print('Test MAE:', mean_absolute_error(y_test, model_1.predict(X_test)))

Baseline Mean Absolute Error :  1201.8811133682555
Test MAE: 1127.2521918670423


In [ ]:
print(df.corr)

<bound method DataFrame.corr of                      bathrooms  bedrooms  ... common_outdoor_space amenities_score
created                                   ...                                     
2016-06-24 07:54:24        1.5         3  ...                    0               0
2016-06-12 12:19:27        1.0         2  ...                    0               5
2016-04-17 03:26:41        1.0         1  ...                    0               3
2016-04-18 02:22:02        1.0         1  ...                    0               2
2016-04-28 01:32:41        1.0         4  ...                    0               1
...                        ...       ...  ...                  ...             ...
2016-06-02 05:41:05        1.0         2  ...                    0               5
2016-04-04 18:22:34        1.0         1  ...                    1               9
2016-04-16 02:13:40        1.0         1  ...                    0               5
2016-04-08 02:13:33        1.0         0  ...          

# LETS ADD BEDROOMS so we have multiple variables


In [ ]:
 
# X_2 = df[['amenities_score', 'bedrooms']]
# y_2 = df['price']

# # New variables because it makes me feel less-nervous. 

# X_2_train, y_2_train = X_2.loc[mask], y_2.loc[mask]
# X_2_test, y_2_test = X_2.loc[~mask], y_2.loc[~mask]


# NOPE. gonna copy from above and change my X at the top. 

In [ ]:
X = df[['amenities_score', 'bedrooms']]
y = df['price']


In [ ]:
cutoff = df.index.month == 6

mask = X.index.month != 6

X_train, y_train = X.loc[mask], y.loc[mask]
X_test, y_test = X.loc[~mask], y.loc[~mask]

In [ ]:
y_pred = [y_train.mean()] * len(y_train) #pretty sure this is my training data
print('Mean Price for an NYC rental from this dataset:', y_train.mean())
print('Baseline Mean Absolute Error : ', mean_absolute_error(y_train,  y_pred))

Mean Price for an NYC rental from this dataset: 3575.604007034292
Baseline Mean Absolute Error :  1201.8811133682555


In [ ]:
# instantiation, as they say. 
model_2 = LinearRegression()

# TRAINING data to the model
model_2.fit(X_train, y_train);

In [ ]:
print('Training MAE:', mean_absolute_error(y_train, model_2.predict(X_train)))
print('Test MAE:', mean_absolute_error(y_test, model_2.predict(X_test)))
print()
print('Training RMSE:', mean_squared_error(y_train, model_2.predict(X_train), squared=False))
print('Test RMSE:', mean_squared_error(y_test, model_2.predict(X_test), squared=False))
print()
print('Training R^2:', r2_score(y_train, model_2.predict(X_train)))
print('Test R^2:', r2_score(y_test, model_2.predict(X_test)))
print()
print('Baseline Mean Absolute Error : ', mean_absolute_error(y_train,  y_pred))
print('Test MAE:', mean_absolute_error(y_test, model_2.predict(X_test)))

Training MAE: 924.2273864082699
Test MAE: 934.4544405067396

Training RMSE: 1424.5184346773515
Test RMSE: 1425.0627136173875

Training R^2: 0.346462373822237
Test R^2: 0.34659271519538093

Baseline Mean Absolute Error :  1201.8811133682555
Test MAE: 934.4544405067396


#Baseline Mean Absolute Error :  1201.8811133682555
#Test MAE: 934.4544405067396 

Patrick and Filipe both got better Test MAE scores than I did. Not by a huge amount, but by at least 100 in Filipe's case. Anyways. Excited for tmw. 

In [ ]:
print('adventures in modeling')

adventures in modeling
